In [55]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("/home/ubuntu/workspace/ImageCaptioning.pytorch")
sys.path.append("/home/ubuntu/workspace/fastai")

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai import *
# import fastai *
# from fastai.rnn_reg import *
# from fastai.rnn_train import *
# from fastai.nlp import *
# from fastai.lm_rnn import *
import torch.utils.model_zoo as model_zoo
import dill as pickle
import torch
from fastai.io import *
from fastai.conv_learner import *
import misc
import pandas as pd

from models import *
from dataloaderraw import *

In [56]:
state_dict = torch.load('/home/ubuntu/workspace/FC/fc-model.pth')

In [57]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def get(self, ele, default):
        return self.__dict__.get(ele, default)

In [58]:
label_path = '/home/ubuntu/workspace/ImageCaptioning.pytorch/data/asin_to_captions.json'
import json
file = open(label_path, encoding='ISO-8859-1')
json_data = json.load(file)
# pd.DataFrame(json_data.items())

In [59]:
a = set()
a.add(1)

In [60]:
min_seen = 5

for key, label in json_data.items():
    json_data[key] = label + " <EOS>"
values = json_data.values()

values = " ".join(list(values))
word_counter = {}

words = values.split(" ")
for w in set(words):
    if w == '':
        next
    word_counter[w] = 0
    
include_words = set()
for word in words:
    word_counter[word] += 1
    if word_counter[word] > min_seen:
        include_words.add(word)
# idx_to_word = { for i, w in  }


In [61]:
len(include_words)

13015

In [62]:
idx_to_word = {i: w for i, w in enumerate(include_words)}
word_to_idx = {w: i for i, w in enumerate(include_words)}
max_words = 15
def label_to_i(word_to_idx, label):
    indexes = []
    for word in label.split(" "):
        unk = '<UNKNOWN>'
        index = word_to_idx.get(word, unk)
        if index != unk:
            indexes.append(index)
    return indexes

In [63]:
label_json = {}
for asin, label in json_data.items():
    label_json[asin] = label_to_i(word_to_idx, label)

In [64]:
longest = 0
longest_i = 0
for k, v in label_json.items():
    if len(v) > longest:
        longest_i = k
        longest = len(v)

In [65]:
batch_size=64

In [66]:
opt = Namespace(
    label_json=label_json,
    idx_to_word=idx_to_word,
    word_to_idx=word_to_idx,
    att_feat_size=2048, 
    att_hid_size=512, 
    batch_size=batch_size, 
    beam_size=2, 
    caption_model='att2in', 
    checkpoint_path='log_a2i', 
    cnn_model='resnet101', 
    coco_json='', 
    current_lr=0.00016384000000000006, 
    drop_prob_lm=0.5, 
    dump_images=1, 
    dump_json=1, 
    dump_path=0, 
    labels_path=label_path,
    fc_feat_size=2048, 
    grad_clip=0.1, 
    id='a2i', 
    image_folder='/home/ubuntu/workspace/ImageCaptioning.pytorch/images/', 
    folder_path='/home/ubuntu/workspace/ImageCaptioning.pytorch/images/',
    valid_path='/home/ubuntu/workspace/ImageCaptioning.pytorch/valid/',
    image_root='', 
    infos_path='../att2in/infos_a2i-best.pkl', 
    input_att_dir='data/cocotalk_att', 
    input_encoding_size=512, 
    input_fc_dir='data/cocotalk_fc', 
    input_json='data/cocotalk.json', 
    input_label_h5='data/cocotalk_label.h5', 
    language_eval=0, learning_rate=0.0005, 
    learning_rate_decay_every=3, 
    learning_rate_decay_rate=0.8, 
    learning_rate_decay_start=0, 
    load_best_score=1, 
    losses_log_every=25, 
    max_epochs=25, 
    model='../att2in/model-best.pth', 
    num_images=2, 
    num_layers=1, 
    optim='adam', 
    optim_alpha=0.9, 
    optim_beta=0.999, 
    optim_epsilon=1e-08, 
    rnn_size=512, 
    rnn_type='lstm', 
    sample_max=1, 
    save_checkpoint_every=3000, 
    scheduled_sampling_increase_every=5, 
    scheduled_sampling_increase_prob=0.05, 
    scheduled_sampling_max_prob=0.25, 
    scheduled_sampling_start=0, 
    self_critical_after=-1, 
    seq_length=30, 
    seq_per_img=5,
    split='test',
    ss_prob=0.15000000000000002,
    temperature=1.0,
    train_only=0,
    val_images_use=5000,
    vocab_size=len(include_words),
    weight_decay=0,
    start_from=None
)

In [67]:
model = FCModel(opt)
# model.load_state_dict(torch.load(os.path.join('/home/ubuntu/workspace/FC/fc-model.pth')))
model.cuda()

FCModel(
  (img_embed): Linear(in_features=2048, out_features=512)
  (core): LSTMCore(
    (i2h): Linear(in_features=512, out_features=2560)
    (h2h): Linear(in_features=512, out_features=2560)
    (dropout): Dropout(p=0.5)
  )
  (embed): Embedding(13016, 512)
  (logit): Linear(in_features=512, out_features=13016)
)

In [68]:
loader = DataLoaderRaw(opt)

DataLoaderRaw loading images from folder:  /home/ubuntu/workspace/ImageCaptioning.pytorch/images/
0
listing all images in directory /home/ubuntu/workspace/ImageCaptioning.pytorch/images/
DataLoaderRaw found  80088  images


In [69]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import numpy as np

import time
import os
from six.moves import cPickle

import opts
import models
from dataloader import *
import eval_utils
import misc.utils as utils

try:
    import tensorflow as tf
except ImportError:
    print("Tensorflow not installed; No tensorboard logging.")
    tf = None

def add_summary_value(writer, key, value, iteration):
    summary = tf.Summary(value=[tf.Summary.Value(tag=key, simple_value=value)])
    writer.add_summary(summary, iteration)

def train(opt):
    opt.use_att = utils.if_use_att(opt.caption_model)
    loader = DataLoaderRaw(opt)
#     opt.vocab_size = loader.vocab_size
#     opt.seq_length = loader.seq_length

    tf_summary_writer = tf and tf.summary.FileWriter(opt.checkpoint_path)

    infos = {}
    histories = {}
    if opt.start_from is not None:
        # open old infos and check if models are compatible
        with open(os.path.join(opt.start_from, 'infos_'+opt.id+'.pkl')) as f:
            infos = cPickle.load(f)
            saved_model_opt = infos['opt']
            need_be_same=["caption_model", "rnn_type", "rnn_size", "num_layers"]
            for checkme in need_be_same:
                assert vars(saved_model_opt)[checkme] == vars(opt)[checkme], "Command line argument and saved model disagree on '%s' " % checkme

        if os.path.isfile(os.path.join(opt.start_from, 'histories_'+opt.id+'.pkl')):
            with open(os.path.join(opt.start_from, 'histories_'+opt.id+'.pkl')) as f:
                histories = cPickle.load(f)

    iteration = infos.get('iter', 0)
    epoch = infos.get('epoch', 0)

    val_result_history = histories.get('val_result_history', {})
    loss_history = histories.get('loss_history', {})
    lr_history = histories.get('lr_history', {})
    ss_prob_history = histories.get('ss_prob_history', {})

#     loader.iterators = infos.get('iterators', loader.iterators)
#     loader.split_ix = infos.get('split_ix', loader.split_ix)
    if opt.load_best_score == 1:
        best_val_score = infos.get('best_val_score', None)

    model = models.setup(opt)
    model.cuda()

    update_lr_flag = True
    # Assure in training mode
    model.train()

    crit = utils.LanguageModelCriterion()

    optimizer = optim.Adam(model.parameters(), lr=opt.learning_rate, weight_decay=opt.weight_decay)

    # Load the optimizer
    if vars(opt).get('start_from', None) is not None:
        optimizer.load_state_dict(torch.load(os.path.join(opt.start_from, 'optimizer.pth')))

    while True:
        if update_lr_flag:
                # Assign the learning rate
            if epoch > opt.learning_rate_decay_start and opt.learning_rate_decay_start >= 0:
                frac = (epoch - opt.learning_rate_decay_start) // opt.learning_rate_decay_every
                decay_factor = opt.learning_rate_decay_rate  ** frac
                opt.current_lr = opt.learning_rate * decay_factor
                utils.set_lr(optimizer, opt.current_lr) # set the decayed rate
            else:
                opt.current_lr = opt.learning_rate
            # Assign the scheduled sampling prob
            if epoch > opt.scheduled_sampling_start and opt.scheduled_sampling_start >= 0:
                frac = (epoch - opt.scheduled_sampling_start) // opt.scheduled_sampling_increase_every
                opt.ss_prob = min(opt.scheduled_sampling_increase_prob  * frac, opt.scheduled_sampling_max_prob)
                model.ss_prob = opt.ss_prob
            update_lr_flag = False
                
        start = time.time()
        # Load data from train split (0)
        data = loader.get_batch('train')
        print('Read data:', time.time() - start)

        torch.cuda.synchronize()
        start = time.time()

        tmp = [data['fc_feats'], data['att_feats'], data['labels']]
        tmp = [Variable(torch.from_numpy(_), requires_grad=False).cuda() for _ in tmp]
        fc_feats, att_feats, labels = tmp
        
        optimizer.zero_grad()
        loss = crit(model(fc_feats, att_feats, labels), labels[:,1:])
        loss.backward()
        utils.clip_gradient(optimizer, opt.grad_clip)
        optimizer.step()
        train_loss = loss.data[0] / batch_size
        torch.cuda.synchronize()
        end = time.time()
        print("iter {} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
            .format(iteration, epoch, train_loss, end - start))

        # Update the iteration and epoch
        iteration += 1
        if data['bounds']['wrapped']:
            epoch += 1
            update_lr_flag = True

        # Write the training loss summary
        if (iteration % opt.losses_log_every == 0):
            if tf is not None:
                add_summary_value(tf_summary_writer, 'train_loss', train_loss, iteration)
                add_summary_value(tf_summary_writer, 'learning_rate', opt.current_lr, iteration)
                add_summary_value(tf_summary_writer, 'scheduled_sampling_prob', model.ss_prob, iteration)
                tf_summary_writer.flush()

            loss_history[iteration] = train_loss
            lr_history[iteration] = opt.current_lr
            ss_prob_history[iteration] = model.ss_prob

        # make evaluation on validation set, and save model
        if (iteration % opt.save_checkpoint_every == 0):
            # eval model
            eval_kwargs = {'split': 'val',
                            'dataset': opt.input_json}
            eval_kwargs.update(vars(opt))
            val_loss, predictions, lang_stats = eval_utils.eval_split(model, crit, loader, eval_kwargs)

            # Write validation result into summary
            if tf is not None:
                add_summary_value(tf_summary_writer, 'validation loss', val_loss, iteration)
                for k,v in lang_stats.items():
                    add_summary_value(tf_summary_writer, k, v, iteration)
                tf_summary_writer.flush()
            val_result_history[iteration] = {'loss': val_loss, 'lang_stats': lang_stats, 'predictions': predictions}

            # Save model if is improving on validation result
            if opt.language_eval == 1:
                current_score = lang_stats['CIDEr']
            else:
                current_score = - val_loss

            best_flag = False
            if True: # if true
                if best_val_score is None or current_score > best_val_score:
                    best_val_score = current_score
                    best_flag = True
                checkpoint_path = os.path.join(opt.checkpoint_path, 'model.pth')
                torch.save(model.state_dict(), checkpoint_path)
                print("model saved to {}".format(checkpoint_path))
                optimizer_path = os.path.join(opt.checkpoint_path, 'optimizer.pth')
                torch.save(optimizer.state_dict(), optimizer_path)

                # Dump miscalleous informations
                infos['iter'] = iteration
                infos['epoch'] = epoch
                infos['iterators'] = loader.iterators
                infos['split_ix'] = loader.split_ix
                infos['best_val_score'] = best_val_score
                infos['opt'] = opt
                infos['vocab'] = loader.get_vocab()

                histories['val_result_history'] = val_result_history
                histories['loss_history'] = loss_history
                histories['lr_history'] = lr_history
                histories['ss_prob_history'] = ss_prob_history
                with open(os.path.join(opt.checkpoint_path, 'infos_'+opt.id+'.pkl'), 'wb') as f:
                    cPickle.dump(infos, f)
                with open(os.path.join(opt.checkpoint_path, 'histories_'+opt.id+'.pkl'), 'wb') as f:
                    cPickle.dump(histories, f)

                if best_flag:
                    checkpoint_path = os.path.join(opt.checkpoint_path, 'model-best.pth')
                    torch.save(model.state_dict(), checkpoint_path)
                    print("model saved to {}".format(checkpoint_path))
                    with open(os.path.join(opt.checkpoint_path, 'infos_'+opt.id+'-best.pkl'), 'wb') as f:
                        cPickle.dump(infos, f)

        # Stop if reaching max epochs
        if epoch >= opt.max_epochs and opt.max_epochs != -1:
            break

Tensorflow not installed; No tensorboard logging.


In [70]:
opt.label_object = json_data

In [71]:
model

DataLoaderRaw loading images from folder:  /home/ubuntu/workspace/ImageCaptioning.pytorch/images/
0
listing all images in directory /home/ubuntu/workspace/ImageCaptioning.pytorch/images/
DataLoaderRaw found  80088  images
Read data: 5.235480308532715


/home/ubuntu/workspace/ImageCaptioning.pytorch/models/Att2inModel.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weight = F.softmax(dot)                             # batch * att_size
/home/ubuntu/workspace/ImageCaptioning.pytorch/models/Att2inModel.py:144: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.log_softmax(self.logit(output))


iter 0 (epoch 0), train_loss = 252.773, time/batch = 0.401
EVALUATING
Read data: 5.305351495742798
iter 1 (epoch 0), train_loss = 173.345, time/batch = 0.424
EVALUATING
Read data: 5.16870641708374
iter 2 (epoch 0), train_loss = 123.652, time/batch = 0.399
EVALUATING
Read data: 5.037405967712402
iter 3 (epoch 0), train_loss = 139.175, time/batch = 0.430
EVALUATING
Read data: 5.025844573974609
iter 4 (epoch 0), train_loss = 121.159, time/batch = 0.446
EVALUATING
Read data: 5.059460401535034
iter 5 (epoch 0), train_loss = 129.613, time/batch = 0.481
EVALUATING
Read data: 4.826989412307739
iter 6 (epoch 0), train_loss = 118.567, time/batch = 0.406
EVALUATING
Read data: 4.968106746673584
iter 7 (epoch 0), train_loss = 120.045, time/batch = 0.428
EVALUATING
Read data: 4.950570344924927
iter 8 (epoch 0), train_loss = 113.053, time/batch = 0.439
EVALUATING
Read data: 5.174998760223389
iter 9 (epoch 0), train_loss = 111.697, time/batch = 0.507
EVALUATING
Read data: 5.140204906463623
iter 10 (ep

Read data: 5.106816530227661
iter 83 (epoch 0), train_loss = 89.259, time/batch = 0.439
EVALUATING
Read data: 5.153535842895508
iter 84 (epoch 0), train_loss = 85.855, time/batch = 0.455
EVALUATING
Read data: 5.166828393936157
iter 85 (epoch 0), train_loss = 77.648, time/batch = 0.415
EVALUATING
Read data: 5.004393577575684
iter 86 (epoch 0), train_loss = 76.256, time/batch = 0.442
EVALUATING
Read data: 4.9457783699035645
iter 87 (epoch 0), train_loss = 77.785, time/batch = 0.359
EVALUATING
Read data: 5.1712424755096436
iter 88 (epoch 0), train_loss = 78.522, time/batch = 0.361
EVALUATING
rescuing
Read data: 7.37626838684082
iter 89 (epoch 0), train_loss = 82.694, time/batch = 0.433
EVALUATING
Read data: 5.152551889419556
iter 90 (epoch 0), train_loss = 82.649, time/batch = 0.403
EVALUATING
Read data: 5.107722043991089
iter 91 (epoch 0), train_loss = 81.059, time/batch = 0.445
EVALUATING
Read data: 5.0071985721588135
iter 92 (epoch 0), train_loss = 84.396, time/batch = 0.417
EVALUATING

Read data: 5.147178888320923
iter 165 (epoch 0), train_loss = 86.370, time/batch = 0.388
EVALUATING
Read data: 5.277986288070679
iter 166 (epoch 0), train_loss = 74.878, time/batch = 0.440
EVALUATING
Read data: 4.986137390136719
iter 167 (epoch 0), train_loss = 81.677, time/batch = 0.533
EVALUATING
Read data: 5.2268385887146
iter 168 (epoch 0), train_loss = 76.058, time/batch = 0.414
EVALUATING
Read data: 5.198153734207153
iter 169 (epoch 0), train_loss = 81.351, time/batch = 0.401
EVALUATING
Read data: 5.240711450576782
iter 170 (epoch 0), train_loss = 72.093, time/batch = 0.415
EVALUATING
Read data: 5.086417198181152
iter 171 (epoch 0), train_loss = 74.289, time/batch = 0.375
EVALUATING
Read data: 5.017797470092773
iter 172 (epoch 0), train_loss = 75.421, time/batch = 0.453
EVALUATING
Read data: 4.868059396743774
iter 173 (epoch 0), train_loss = 77.905, time/batch = 0.470
EVALUATING
Read data: 5.064418315887451
iter 174 (epoch 0), train_loss = 73.906, time/batch = 0.434
EVALUATING
Re

...

Read data: 5.130650997161865
iter 1311 (epoch 1), train_loss = 61.209, time/batch = 0.524
EVALUATING
Read data: 5.064707279205322
iter 1312 (epoch 1), train_loss = 72.707, time/batch = 0.442
EVALUATING
Read data: 5.191277265548706
iter 1313 (epoch 1), train_loss = 69.173, time/batch = 0.440
EVALUATING
Read data: 5.31105637550354
iter 1314 (epoch 1), train_loss = 66.983, time/batch = 0.378
EVALUATING
Read data: 4.90643835067749
iter 1315 (epoch 1), train_loss = 68.759, time/batch = 0.454
EVALUATING
Read data: 5.144270420074463
iter 1316 (epoch 1), train_loss = 64.618, time/batch = 0.375
EVALUATING
Read data: 5.290613412857056
iter 1317 (epoch 1), train_loss = 68.521, time/batch = 0.440
EVALUATING
Read data: 5.111786842346191
iter 1318 (epoch 1), train_loss = 71.288, time/batch = 0.429
EVALUATING
Read data: 5.199856281280518
iter 1319 (epoch 1), train_loss = 73.256, time/batch = 0.485
EVALUATING
Read data: 5.172048568725586
iter 1320 (epoch 1), train_loss = 67.163, time/batch = 0.445
EVA

iter 1392 (epoch 1), train_loss = 62.137, time/batch = 0.421
EVALUATING
Read data: 5.10453462600708
iter 1393 (epoch 1), train_loss = 82.074, time/batch = 0.469
EVALUATING
Read data: 5.1506736278533936
iter 1394 (epoch 1), train_loss = 61.676, time/batch = 0.414
EVALUATING
Read data: 5.0653533935546875
iter 1395 (epoch 1), train_loss = 70.961, time/batch = 0.461
EVALUATING
Read data: 5.00235652923584
iter 1396 (epoch 1), train_loss = 72.661, time/batch = 0.428
EVALUATING
Read data: 4.9617085456848145
iter 1397 (epoch 1), train_loss = 61.417, time/batch = 0.328
EVALUATING
Read data: 5.1158127784729
iter 1398 (epoch 1), train_loss = 66.771, time/batch = 0.441
EVALUATING
Read data: 5.2822253704071045
iter 1399 (epoch 1), train_loss = 65.546, time/batch = 0.479
EVALUATING
Read data: 5.303804159164429
iter 1400 (epoch 1), train_loss = 64.584, time/batch = 0.430
EVALUATING
Read data: 5.080500602722168
iter 1401 (epoch 1), train_loss = 72.712, time/batch = 0.427
EVALUATING
Read data: 5.209630

Read data: 5.300338983535767
iter 1474 (epoch 1), train_loss = 68.621, time/batch = 0.483
EVALUATING
Read data: 5.204888582229614
iter 1475 (epoch 1), train_loss = 72.934, time/batch = 0.466
EVALUATING
Read data: 5.259723424911499
iter 1476 (epoch 1), train_loss = 66.316, time/batch = 0.476
EVALUATING
Read data: 5.1887431144714355
iter 1477 (epoch 1), train_loss = 69.320, time/batch = 0.483
EVALUATING
Read data: 5.088981628417969
iter 1478 (epoch 1), train_loss = 75.929, time/batch = 0.539
EVALUATING
Read data: 4.953866243362427
iter 1479 (epoch 1), train_loss = 76.194, time/batch = 0.478
EVALUATING
Read data: 5.2458343505859375
iter 1480 (epoch 1), train_loss = 68.532, time/batch = 0.481
EVALUATING
Read data: 5.214127779006958
iter 1481 (epoch 1), train_loss = 63.085, time/batch = 0.494
EVALUATING
Read data: 5.296900272369385
iter 1482 (epoch 1), train_loss = 65.859, time/batch = 0.497
EVALUATING
Read data: 5.0610268115997314
iter 1483 (epoch 1), train_loss = 64.347, time/batch = 0.38

Read data: 5.28473973274231
iter 1555 (epoch 1), train_loss = 70.601, time/batch = 0.477
EVALUATING
Read data: 5.348208904266357
iter 1556 (epoch 1), train_loss = 67.590, time/batch = 0.511
EVALUATING
Read data: 5.047255277633667
iter 1557 (epoch 1), train_loss = 59.634, time/batch = 0.481
EVALUATING
Read data: 5.245269775390625
iter 1558 (epoch 1), train_loss = 71.939, time/batch = 0.469
EVALUATING
Read data: 5.180579900741577
iter 1559 (epoch 1), train_loss = 67.883, time/batch = 0.467
EVALUATING
Read data: 5.145096302032471
iter 1560 (epoch 1), train_loss = 67.064, time/batch = 0.468
EVALUATING
Read data: 5.193490982055664
iter 1561 (epoch 1), train_loss = 64.112, time/batch = 0.413
EVALUATING
Read data: 5.0354249477386475
iter 1562 (epoch 1), train_loss = 71.372, time/batch = 0.431
EVALUATING
Read data: 5.160248041152954
iter 1563 (epoch 1), train_loss = 53.749, time/batch = 0.376
EVALUATING
Read data: 5.1528000831604
iter 1564 (epoch 1), train_loss = 68.651, time/batch = 0.416
EVA

TypeError: float() argument must be a string or a number, not 'JpegImageFile'

In [72]:
torch.save(model.state_dict(), '/home/ubuntu/workspace/ImageCaptioning.pytorch/data/model.pth')

In [165]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import numpy as np

import time
import os
from six.moves import cPickle

import opts
import models
from dataloader import *
import eval_utils
import misc.utils as utils

try:
    import tensorflow as tf
except ImportError:
    print("Tensorflow not installed; No tensorboard logging.")
    tf = None

def add_summary_value(writer, key, value, iteration):
    summary = tf.Summary(value=[tf.Summary.Value(tag=key, simple_value=value)])
    writer.add_summary(summary, iteration)

def test(opt):
    opt.use_att = utils.if_use_att(opt.caption_model)
    loader = DataLoaderRaw(opt)
#     opt.vocab_size = loader.vocab_size
#     opt.seq_length = loader.seq_length

    tf_summary_writer = tf and tf.summary.FileWriter(opt.checkpoint_path)

    infos = {}
    histories = {}
    if opt.start_from is not None:
        # open old infos and check if models are compatible
        with open(os.path.join(opt.start_from, 'infos_'+opt.id+'.pkl')) as f:
            infos = cPickle.load(f)
            saved_model_opt = infos['opt']
            need_be_same=["caption_model", "rnn_type", "rnn_size", "num_layers"]
            for checkme in need_be_same:
                assert vars(saved_model_opt)[checkme] == vars(opt)[checkme], "Command line argument and saved model disagree on '%s' " % checkme

        if os.path.isfile(os.path.join(opt.start_from, 'histories_'+opt.id+'.pkl')):
            with open(os.path.join(opt.start_from, 'histories_'+opt.id+'.pkl')) as f:
                histories = cPickle.load(f)

    iteration = infos.get('iter', 0)
    epoch = infos.get('epoch', 0)

    val_result_history = histories.get('val_result_history', {})
    loss_history = histories.get('loss_history', {})
    lr_history = histories.get('lr_history', {})
    ss_prob_history = histories.get('ss_prob_history', {})

#     loader.iterators = infos.get('iterators', loader.iterators)
#     loader.split_ix = infos.get('split_ix', loader.split_ix)
    if opt.load_best_score == 1:
        best_val_score = infos.get('best_val_score', None)

#     model = models.setup(opt)
#     model.cuda()

    update_lr_flag = True
    # Assure in training mode
#     model.train()

    crit = utils.LanguageModelCriterion()

    optimizer = optim.Adam(model.parameters(), lr=opt.learning_rate, weight_decay=opt.weight_decay)

    # Load the optimizer
    if vars(opt).get('start_from', None) is not None:
        optimizer.load_state_dict(torch.load(os.path.join(opt.start_from, 'optimizer.pth')))

    while True:
        if update_lr_flag:
                # Assign the learning rate
            if epoch > opt.learning_rate_decay_start and opt.learning_rate_decay_start >= 0:
                frac = (epoch - opt.learning_rate_decay_start) // opt.learning_rate_decay_every
                decay_factor = opt.learning_rate_decay_rate  ** frac
                opt.current_lr = opt.learning_rate * decay_factor
                utils.set_lr(optimizer, opt.current_lr) # set the decayed rate
            else:
                opt.current_lr = opt.learning_rate
            # Assign the scheduled sampling prob
            if epoch > opt.scheduled_sampling_start and opt.scheduled_sampling_start >= 0:
                frac = (epoch - opt.scheduled_sampling_start) // opt.scheduled_sampling_increase_every
                opt.ss_prob = min(opt.scheduled_sampling_increase_prob  * frac, opt.scheduled_sampling_max_prob)
                model.ss_prob = opt.ss_prob
            update_lr_flag = False
                
        start = time.time()
        # Load data from train split (0)
        data = loader.get_batch('test')
        print('Read data:', time.time() - start)

        torch.cuda.synchronize()
        start = time.time()

        tmp = [data['fc_feats'], data['att_feats'], data['labels']]
        tmp = [Variable(torch.from_numpy(_), requires_grad=False).cuda() for _ in tmp]
        fc_feats, att_feats, labels = tmp
        
        optimizer.zero_grad()
        loss = crit(model(fc_feats, att_feats, labels), labels[:,1:])
        loss.backward()
        utils.clip_gradient(optimizer, opt.grad_clip)
        optimizer.step()
        train_loss = loss.data[0] / batch_size
        torch.cuda.synchronize()
        end = time.time()
        print("iter {} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
            .format(iteration, epoch, train_loss, end - start))

        # Update the iteration and epoch
        iteration += 1
        if data['bounds']['wrapped']:
            epoch += 1
            update_lr_flag = True

        # Write the training loss summary
        if (iteration % opt.losses_log_every == 0):
            if tf is not None:
                add_summary_value(tf_summary_writer, 'train_loss', train_loss, iteration)
                add_summary_value(tf_summary_writer, 'learning_rate', opt.current_lr, iteration)
                add_summary_value(tf_summary_writer, 'scheduled_sampling_prob', model.ss_prob, iteration)
                tf_summary_writer.flush()

            loss_history[iteration] = train_loss
            lr_history[iteration] = opt.current_lr
            ss_prob_history[iteration] = model.ss_prob

        # make evaluation on validation set, and save model
        if (iteration % opt.save_checkpoint_every == 0):
            # eval model
            eval_kwargs = {'split': 'val',
                            'dataset': opt.input_json}
            eval_kwargs.update(vars(opt))
            val_loss, predictions, lang_stats = eval_utils.eval_split(model, crit, loader, eval_kwargs)

            # Write validation result into summary
            if tf is not None:
                add_summary_value(tf_summary_writer, 'validation loss', val_loss, iteration)
                for k,v in lang_stats.items():
                    add_summary_value(tf_summary_writer, k, v, iteration)
                tf_summary_writer.flush()
            val_result_history[iteration] = {'loss': val_loss, 'lang_stats': lang_stats, 'predictions': predictions}

            # Save model if is improving on validation result
            if opt.language_eval == 1:
                current_score = lang_stats['CIDEr']
            else:
                current_score = - val_loss

            best_flag = False
            if True: # if true
                if best_val_score is None or current_score > best_val_score:
                    best_val_score = current_score
                    best_flag = True
                checkpoint_path = os.path.join(opt.checkpoint_path, 'model.pth')
                torch.save(model.state_dict(), checkpoint_path)
                print("model saved to {}".format(checkpoint_path))
                optimizer_path = os.path.join(opt.checkpoint_path, 'optimizer.pth')
                torch.save(optimizer.state_dict(), optimizer_path)

                # Dump miscalleous informations
                infos['iter'] = iteration
                infos['epoch'] = epoch
                infos['iterators'] = loader.iterators
                infos['split_ix'] = loader.split_ix
                infos['best_val_score'] = best_val_score
                infos['opt'] = opt
                infos['vocab'] = loader.get_vocab()

                histories['val_result_history'] = val_result_history
                histories['loss_history'] = loss_history
                histories['lr_history'] = lr_history
                histories['ss_prob_history'] = ss_prob_history
                with open(os.path.join(opt.checkpoint_path, 'infos_'+opt.id+'.pkl'), 'wb') as f:
                    cPickle.dump(infos, f)
                with open(os.path.join(opt.checkpoint_path, 'histories_'+opt.id+'.pkl'), 'wb') as f:
                    cPickle.dump(histories, f)

                if best_flag:
                    checkpoint_path = os.path.join(opt.checkpoint_path, 'model-best.pth')
                    torch.save(model.state_dict(), checkpoint_path)
                    print("model saved to {}".format(checkpoint_path))
                    with open(os.path.join(opt.checkpoint_path, 'infos_'+opt.id+'-best.pkl'), 'wb') as f:
                        cPickle.dump(infos, f)

        # Stop if reaching max epochs
        if epoch >= opt.max_epochs and opt.max_epochs != -1:
            break

array([[ 1.,  2.,  3.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [77]:
data = loader.get_batch('val')
optimizer = optim.Adam(model.parameters(), lr=opt.learning_rate, weight_decay=opt.weight_decay)

# Load the optimizer
if vars(opt).get('start_from', None) is not None:
    optimizer.load_state_dict(torch.load(os.path.join(opt.start_from, 'optimizer.pth')))
tmp = [data['fc_feats'], data['att_feats'], data['labels']]
tmp = [Variable(torch.from_numpy(_), requires_grad=False).cuda() for _ in tmp]
fc_feats, att_feats, labels = tmp

In [78]:
model.eval()
# torch.cuda.synchronize()


FCModel(
  (img_embed): Linear(in_features=2048, out_features=512)
  (core): LSTMCore(
    (i2h): Linear(in_features=512, out_features=2560)
    (h2h): Linear(in_features=512, out_features=2560)
    (dropout): Dropout(p=0.5)
  )
  (embed): Embedding(13016, 512)
  (logit): Linear(in_features=512, out_features=13016)
)

In [79]:
crit = utils.LanguageModelCriterion()


In [157]:
data_loader = DataLoaderRaw(opt)
data = loader.get_batch('train')

DataLoaderRaw loading images from folder:  /home/ubuntu/workspace/ImageCaptioning.pytorch/images/
0
listing all images in directory /home/ubuntu/workspace/ImageCaptioning.pytorch/images/
DataLoaderRaw found  80088  images


In [158]:
tmp = [data['fc_feats'], data['att_feats'], data['labels']]
tmp = [Variable(torch.from_numpy(_), requires_grad=False).cuda() for _ in tmp]
fc_feats, att_feats, labels = tmp

optimizer.zero_grad()
results = model(fc_feats, att_feats, labels)
loss = crit(results, labels[:,1:])

/home/ubuntu/workspace/ImageCaptioning.pytorch/models/FCModel.py:111: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.log_softmax(self.logit(output))


In [159]:
seq, _ = model.sample(fc_feats,att_feats)


/home/ubuntu/workspace/ImageCaptioning.pytorch/models/FCModel.py:198: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logprobs = F.log_softmax(self.logit(output))


In [160]:
len(seq[0])

30

In [161]:
fc_feats

Variable containing:
  1.7569   0.2160   0.2002  ...    0.6460   0.0493   0.0862
  0.5674   0.0000   0.0000  ...    0.0226   0.0621   0.0165
  0.2495   0.0069   0.1452  ...    0.0000   0.1048   0.0418
           ...               ⋱              ...            
  1.2170   0.0457   0.0000  ...    0.0531   0.0029   0.0024
  0.5394   0.0687   0.0408  ...    0.0447   0.1923   0.0514
  1.7932   0.0255   0.0869  ...    0.4511   0.0259   0.0122
[torch.cuda.FloatTensor of size 64x2048 (GPU 0)]

In [162]:
labels = []
for label in data['labels']:
    words = ""
    for i in label:
        words += idx_to_word.get(i, "UNKNOWN") + " "
    labels.append(words)

In [163]:
labels_prediction = []
for label in seq:
    words = set()
    for i in label:
        words.add(idx_to_word[i])
    labels_prediction.append(" ".join(words))

In [167]:
data['asins_to_label'].keys()

dict_keys(['B003F6EI2C', 'B01IQ60ZFE', 'B075V3WC2F', 'B06XFPQ3ZN', 'B0071URUW8', 'B0759MW7L3', 'B01M15PLEH', 'B01N43BBIO', 'B01H6Y1DM6', 'B000F3FVTC', 'B00PJIQ5A2', 'B007SNADYA', 'B00MR9T9ZY', 'B074W5LM1K', 'B06Y6HKCWW', 'B014W1C262', 'B00MC7TC6M', 'B075MBGLS1', 'B072JTX7KK', 'B00AGNMRIO', 'B00Q7DETQU', 'B00S6ZDFJE', 'B0778WNJTT', 'B01AV86K72', 'B01N5F1OM3', 'B00SX5ULXK', 'B01ES9VFF2', 'B0081SKI28', 'B0761SG5DN', 'B017PEGG0G', 'B01HR215FM', 'B074PV1GJM', 'B06XTQJ1BV', 'B01M0SASZO', 'B01MY0AZHI', 'B01J4EIRTI', 'B075R13JLG', 'B01M098VFG', 'B00ZBJVH9I', 'B01ISL7P6E', 'B075MWDHSF', 'B06XSKSPLW', 'B011TR7FSC', 'B076J9HQBP', 'B075HTQZ26', 'B071GP64JM', 'B0761SCQZ7', 'B00X6L4J70', 'B073HPCH9D', 'B00NARZQKY', 'B01N5BNROP', 'B075BNFRGP', 'B01N29Q7LA', 'B00PVLHHRC', 'B01N2W2BMW', 'B07476XJC7', 'B00BJOGIDE', 'B076VGYX9H', 'B071H476Z4', 'B071LPWM11', 'B074TKPNQZ', 'B01N31DPE5', 'B001TJWZPY', 'B072P6CTV4'])

In [164]:
labels_prediction

['Pattern, Tinted Smile Cynthia Liquid 2.0 18.5 EXCEED Market Saddle',
 'McAlister Tinted Smile are (BLACK) Cynthia 2.0 18.5 Mount Saddle HNNATTA',
 'Pattern, Tinted Knot, Sparkling Cynthia 2.0 Spatula Height 18.5 BGSD',
 'McAlister Tinted Smile (BLACK) Cynthia 2.0 Underpants 18.5 15-Ounce Saddle',
 'Underpants Port, Sparkling Smile',
 'McAlister Smile Sparkling Underpants Port, 15-Ounce',
 'Shears Multi-Color Smile Tinted 8-Inch grinderPUNCH 2.0 Cynthia Height 18.5 EXCEED Market Saddle',
 'Anne Tinted Cynthia 2.0 Underpants 18.5 not Saddle',
 'Madonna Smile 18X18 Underpants Port, 15-Ounce',
 'Pattern, Smile Sparkling Underpants Port, 15-Ounce',
 'Curb Pattern, Joseph Tinted Cynthia Square) 2.0 18.5',
 'Puzzle Port, Smile',
 'Pattern, Tinted Cynthia 2.0 (2, 18.5 Hosiery',
 'Shears best Tinted Smile 8-Inch Cynthia 2.0 (2, 18.5 EXCEED Hosiery BIA Market Astylish Saddle',
 'Smile Boys Underpants Port, 15-Ounce',
 '15-Ounce Port, Patina Smile',
 'Tinted Smile Cynthia 2.0 Height 18.5 EXCEED

In [165]:
labels

['Prodyne Prodyne Acrylic Salad Bowl with Servers <EOS>                                                                    ',
 '22 Spacious Durable Large Canvas Tote Bag Pocket Pool Beach Shopping Travel Tote Bag Eco-Friendly (1, <EOS>                                                           ',
 'Women Hat Lace Head Cover Bonnet Cap Scarf Muslim Hat <EOS>                                                                 ',
 'Aluminum 1.2L Outdoor Camping Hiking Kettle Coffee Pot Portable Teapot Kettle, Compact and Lightweight with Silicon Handle <EOS>                                                          ',
 'Brand New World Upholstered Toddler Sofa <EOS>                                                                     ',
 "Cambridge Select Women's Classic Round Toe Mid Heel Dress Pump <EOS>                                                                 ",
 'MODERNO Mens Checkered Shirts Dress Slim Fit Long Sleeve Button down <EOS>                                                 

In [153]:
from IPython.display import Image
from IPython.core.display import HTML 
img = Image(url='/home/ubuntu/workspace/ImageCaptioning.pytorch/images/B01FTCX14A.jpg')

display(img)

In [95]:

results[:][:][0].shape

torch.Size([27, 13016])